In [2]:
import time

# coding: utf-8
import requests

def get_candle_sticks(pair: str, after: str = '', before: str = '', interval: str = '7d'):
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

    url = '/spot/candlesticks'
    query_param = f'currency_pair={pair}&interval={interval}'
    if after != '':
        query_param += f'&from={after}'
    if before != '':
        query_param += f'&to={before}'

    r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
    return r.json()

In [7]:
cs = get_candle_sticks(pair='BTC_USDT')

print(cs)

[['1638748800', '867634366.75980400', '50060.6', '51917.99', '46772.3', '49395.73', '17651.91120781'], ['1639353600', '685901491.92981140', '46691.75', '50183.44', '45455.3', '50058.12', '14463.37598711'], ['1639958400', '494501836.71176106', '50769.1', '51816.08', '45573.97', '46683.15', '10051.09464372'], ['1640563200', '759599244.50330850', '47296.37', '52079.14', '45700', '50769.1', '15806.54176038'], ['1641168000', '892842910.31189970', '41860.17', '47564.23', '40528.06', '47296.36', '20524.01574458'], ['1641772800', '522086714.12299454', '43072.57', '44418.93', '39653.46', '41868.74', '12310.30745621'], ['1642377600', '619895007.03345980', '36254.2', '43495', '34009.13', '43072.58', '15825.95065721'], ['1642982400', '861560544.44439350', '37893.1', '38888.88', '32944.17', '36254.2', '23563.84332420'], ['1643587200', '672788363.88937700', '42368.01', '42661.74', '36250.17', '37888.01', '17332.66196211'], ['1644192000', '626701614.65755140', '42061.38', '45816.6', '41612.03', '4238

根据与gateio的tradingview对照，返回的字段应该是：['时间戳', 'xxx', '收盘', '高点', '低点', '开盘', '成交量']

In [3]:
import datetime

def get_time(timestamp: str):
    dt_object = datetime.datetime.fromtimestamp(int(timestamp))
    return dt_object

In [15]:
def get_closing_price(pair: str):
    week_closing_price = []

    candle_sticks = get_candle_sticks(pair)
    for item in candle_sticks[::-1]:
        week_closing_price.append([item[0], item[2]])

    if len(candle_sticks) == 0:
        return week_closing_price;

    while True:
        candle_sticks = get_candle_sticks(pair, before=week_closing_price[-1][0])

        for item in candle_sticks[::-1]:
            week_closing_price.append([item[0], item[2]])

        if len(candle_sticks) < 100:
            break

    print(len(week_closing_price))
    print(get_time(week_closing_price[-1][0]))
    return week_closing_price

In [10]:
BTC_USDT = get_closing_price('BTC_USDT')


560
2013-03-25 08:00:00


In [21]:
ETH_USDT = get_closing_price('ETH_USDT')

405
2016-03-07 08:00:00


In [22]:
SOL_USDT = get_closing_price('SOL_USDT')

178
2020-06-22 08:00:00


In [5]:
def get_all_pairs():
    host = "https://api.gateio.ws"
    prefix = "/api/v4"
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

    url = '/spot/currency_pairs'
    query_param = ''
    r = requests.request('GET', host + prefix + url, headers=headers)
    return r.json()

In [6]:
all_pairs = get_all_pairs()
# print(all_pairs)
print(len(all_pairs))

all_pairs = [item['id'] for item in all_pairs]
print(all_pairs)

3157
['10SET_USDT', '1ART_USDT', '1EARTH_USDT', '1INCH3L_USDT', '1INCH3S_USDT', '1INCH_ETH', '1INCH_TRY', '1INCH_USDT', '3KM_USDT', '4TOKEN_USDT', '88MPH_ETH', '88MPH_USDT', 'AAG_USDT', 'AART_ETH', 'AART_USDT', 'AAVE3L_USDT', 'AAVE3S_USDT', 'AAVE_ETH', 'AAVE_TRY', 'AAVE_USDT', 'ABBC_USDT', 'ABT_ETH', 'ABT_USDT', 'ACA_USDT', 'ACE_USDT', 'ACH3L_USDT', 'ACH3S_USDT', 'ACH_ETH', 'ACH_USDT', 'ACM_TRY', 'ACM_USDT', 'ACN_USDT', 'ACS_USDT', 'ACX_USDT', 'ADA3L_USDT', 'ADA3S_USDT', 'ADAPAD_USDT', 'ADA_BTC', 'ADA_TRY', 'ADA_USDT', 'ADD_ETH', 'ADEL_ETH', 'ADEL_USDT', 'ADP_ETH', 'ADP_USDT', 'ADS_USDT', 'ADX_ETH', 'ADX_USDT', 'AD_USDT', 'AERGO_ETH', 'AERGO_USDT', 'AE_BTC', 'AE_ETH', 'AE_USDT', 'AFC_USDT', 'AGB_USDT', 'AGILITY_USDT', 'AGIX_USDT', 'AGI_USDT', 'AGLA_USDT', 'AGLD_ETH', 'AGLD_USDT', 'AGS_USDT', 'AG_USDT', 'AIBB_USDT', 'AICODE_USDT', 'AIDOGE_TRY', 'AIDOGE_USDT', 'AIEPK_USDT', 'AIE_USDT', 'AIOZ_ETH', 'AIOZ_USDT', 'AIRTNT_USDT', 'AIR_USDT', 'AISC_USDT', 'AIS_USDT', 'AITECH_USDT', 'AKITA_USDT

In [7]:
# 创建一个空字典来存储分类列表
pairs = {}

# 遍历inst_ids列表
for inst_id in all_pairs:
    # 获取yy部分
    stable = inst_id.split('_')[1]
    # 将inst_id添加到对应分类的列表中
    if stable in pairs:
        pairs[stable].append(inst_id)
    else:
        pairs[stable] = [inst_id]

pairs = {k: sorted(v) for k, v in pairs.items()}

# 打印分类列表
for category, values in pairs.items():
    print(f"Pairs {category}: {values}")

Pairs USDT: ['10SET_USDT', '1ART_USDT', '1EARTH_USDT', '1INCH3L_USDT', '1INCH3S_USDT', '1INCH_USDT', '3KM_USDT', '4TOKEN_USDT', '88MPH_USDT', 'AAG_USDT', 'AART_USDT', 'AAVE3L_USDT', 'AAVE3S_USDT', 'AAVE_USDT', 'ABBC_USDT', 'ABT_USDT', 'ACA_USDT', 'ACE_USDT', 'ACH3L_USDT', 'ACH3S_USDT', 'ACH_USDT', 'ACM_USDT', 'ACN_USDT', 'ACS_USDT', 'ACX_USDT', 'ADA3L_USDT', 'ADA3S_USDT', 'ADAPAD_USDT', 'ADA_USDT', 'ADEL_USDT', 'ADP_USDT', 'ADS_USDT', 'ADX_USDT', 'AD_USDT', 'AERGO_USDT', 'AE_USDT', 'AFC_USDT', 'AGB_USDT', 'AGILITY_USDT', 'AGIX_USDT', 'AGI_USDT', 'AGLA_USDT', 'AGLD_USDT', 'AGS_USDT', 'AG_USDT', 'AIBB_USDT', 'AICODE_USDT', 'AIDOGE_USDT', 'AIEPK_USDT', 'AIE_USDT', 'AIOZ_USDT', 'AIRTNT_USDT', 'AIR_USDT', 'AISC_USDT', 'AIS_USDT', 'AITECH_USDT', 'AKITA_USDT', 'AKRO_USDT', 'AKT_USDT', 'ALAYA_USDT', 'ALA_USDT', 'ALCX_USDT', 'ALD_USDT', 'ALEPH_USDT', 'ALEX_USDT', 'ALGO3L_USDT', 'ALGO3S_USDT', 'ALGO_USDT', 'ALICE3L_USDT', 'ALICE3S_USDT', 'ALICE_USDT', 'ALI_USDT', 'ALN_USDT', 'ALPACA_USDT', 'ALPA

In [27]:
usdt_pairs = pairs['USDT']
print(len(usdt_pairs))

2081


In [8]:
import re

# 定义要匹配的模式
pattern = re.compile(r'(3S|3L)$')

x3_pairs, clean_pairs = [], []
# 去除三倍多/三倍空
for pair in pairs['USDT']:
    token, _ = pair.split('_')
    if re.search(pattern, token):
        x3_pairs.append(pair)
    else:
        clean_pairs.append(pair)


print(clean_pairs)

['10SET_USDT', '1ART_USDT', '1EARTH_USDT', '1INCH_USDT', '3KM_USDT', '4TOKEN_USDT', '88MPH_USDT', 'AAG_USDT', 'AART_USDT', 'AAVE_USDT', 'ABBC_USDT', 'ABT_USDT', 'ACA_USDT', 'ACE_USDT', 'ACH_USDT', 'ACM_USDT', 'ACN_USDT', 'ACS_USDT', 'ACX_USDT', 'ADAPAD_USDT', 'ADA_USDT', 'ADEL_USDT', 'ADP_USDT', 'ADS_USDT', 'ADX_USDT', 'AD_USDT', 'AERGO_USDT', 'AE_USDT', 'AFC_USDT', 'AGB_USDT', 'AGILITY_USDT', 'AGIX_USDT', 'AGI_USDT', 'AGLA_USDT', 'AGLD_USDT', 'AGS_USDT', 'AG_USDT', 'AIBB_USDT', 'AICODE_USDT', 'AIDOGE_USDT', 'AIEPK_USDT', 'AIE_USDT', 'AIOZ_USDT', 'AIRTNT_USDT', 'AIR_USDT', 'AISC_USDT', 'AIS_USDT', 'AITECH_USDT', 'AKITA_USDT', 'AKRO_USDT', 'AKT_USDT', 'ALAYA_USDT', 'ALA_USDT', 'ALCX_USDT', 'ALD_USDT', 'ALEPH_USDT', 'ALEX_USDT', 'ALGO_USDT', 'ALICE_USDT', 'ALI_USDT', 'ALN_USDT', 'ALPACA_USDT', 'ALPA_USDT', 'ALPHA_USDT', 'ALPHR_USDT', 'ALPH_USDT', 'ALPINE_USDT', 'ALTB_USDT', 'ALU_USDT', 'ALY_USDT', 'AMB_USDT', 'AMC_USDT', 'AME_USDT', 'AMKT_USDT', 'AMPL_USDT', 'AMP_USDT', 'AMS_USDT', 'AM_U

In [16]:
import time

closing_price = {}
for pair in clean_pairs:
    try:
        closing_price[pair] = get_closing_price(pair)
    except Exception as e:
        time.sleep(1)
        closing_price[pair] = get_closing_price(pair)

129
2021-05-31 08:00:00
108
2021-10-25 08:00:00
90
2022-02-28 08:00:00
152
2020-12-21 08:00:00
31
2023-04-17 08:00:00
26
2023-05-22 08:00:00
154
2020-12-07 08:00:00
100
2021-12-20 08:00:00
99
2021-12-27 08:00:00
163
2020-10-05 08:00:00
156
2020-11-23 08:00:00
302
2018-02-19 08:00:00
95
2022-01-24 08:00:00
105
2021-11-15 08:00:00
153
2020-12-14 08:00:00
62
2022-09-12 08:00:00
17
2023-07-24 08:00:00
40
2023-02-13 08:00:00
51
2022-11-28 08:00:00
105
2021-11-15 08:00:00
311
2017-12-18 08:00:00
168
2020-08-31 08:00:00
92
2022-02-14 08:00:00
94
2022-01-31 08:00:00
138
2021-03-29 08:00:00
31
2023-04-17 08:00:00
142
2021-03-01 08:00:00
317
2017-11-06 08:00:00
86
2022-03-28 08:00:00
20
2023-07-03 08:00:00
31
2023-04-17 08:00:00
42
2023-01-30 08:00:00
32
2023-04-10 08:00:00
39
2023-02-20 08:00:00
116
2021-08-30 08:00:00
168
2020-08-31 08:00:00
22
2023-06-19 08:00:00
14
2023-08-14 08:00:00
29
2023-05-01 08:00:00
31
2023-04-17 08:00:00
14
2023-08-14 08:00:00
17
2023-07-24 08:00:00
137
2021-04-05 0

ProxyError: HTTPSConnectionPool(host='api.gateio.ws', port=443): Max retries exceeded with url: /api/v4/spot/candlesticks?currency_pair=KP3R_USDT&interval=7d&to=1638748800 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))

In [18]:
import json

# 将字典数据保存到文件中
with open('./data.json', 'w') as f:
    json.dump(closing_price, f, indent=4)

print("数据已保存到文件中。")


数据已保存到文件中。
